# Using TINC Client to interact with a C++ application

This notebook demonstrates the use of TincClient to connect to a TincServer hosted in a C++ application. This example uses CASM viewer: https://github.com/AlloSphere-Research-Group/casm_viewer

In [1]:
from tinc import *
TincVersion()

0.9


In [2]:
tclient = TincClient()

Connected to localhost:34450. Server version 1 revision 0


In [3]:
[(p.id, p.group) for p in tclient.get_parameters()]

[('ShellSiteTypes', ''),
 ('PercolationTypes', ''),
 ('markerColor', ''),
 ('markerScale', ''),
 ('percoMarkerScaleFactor', ''),
 ('currentSelection', ''),
 ('previousSelection', ''),
 ('dataset', ''),
 ('alpha', 'history'),
 ('width', 'history'),
 ('alpha', 'trajectory'),
 ('width', 'trajectory'),
 ('time', ''),
 ('dir', ''),
 ('T', ''),
 ('param_chem_pot(a)', '')]

In [4]:
time = tclient.get_parameter("time")

In [5]:
time.value = 100

Setting the value from python will alter the parameter in C++ and vice versa.

In [6]:
time.value

1300

Changes van trigger any computation tied to that parameter. Changes in the parameter in C++ will be forwarded to python and trigger computation

In [7]:
def time_callback(value):
    print(f"Time changed to {value}")

time.register_callback(time_callback)

In [8]:
time.value = 150

Time changed to 150
Time changed to 787


TINC's caching system can be used together with these callbacks or through the ParameterSpace class.

Parameters can have more complex data than a single float value. For example, CASM viewer exposes the color of the atom marker, which can be set from python:

In [9]:
tclient.get_parameter("markerColor").value = [1,0,0, 1]
tclient.get_parameter("markerScale").value = 4

In [10]:
tclient.get_parameter("markerColor").value = [1,1,0, 1]
tclient.get_parameter("markerScale").value = 2

We have used a 'ParameterChoice' to display the shell site types. A ParameterChoice is a bitfield associated with a list of strings

# Data exchange

Parameters values can be exchanged through the network, but it is often necessary to exchange data that is both large and complex. TINC provides DiskBuffers for this. The exchange is seamless and reactive, and there are higher level classes in TINC that handle loading and decoding of the data as well as classes that connect disk buffers to display elements. Here are two examples of these implemented in CASM viewer.

## Image disk buffer

Image disk buffers can be used to create images in python that are displayed in the C++ application.

In [11]:
[db.id for db in tclient.disk_buffers]

['graph',
 'graph1',
 'graph2',
 'graph3',
 'graph4',
 'graph5',
 'history_buffer',
 'trajectory_buffer']

In [12]:
imageBuffer = tclient.get_disk_buffer('graph1')

In [13]:
import random
import matplotlib.pyplot as plt
import threading

def write_graph(parameter_value):
    data = [random.random() * parameter_value for i in range(10)]
    with threading.Lock():
        fname = imageBuffer.get_filename_for_writing()
        f = plt.figure()
        plt.title("Random numbers with range 0->" + str(parameter_value))
        plt.plot(data)
        plt.savefig(fname)
        plt.close()
    f.clf()
    del f
    imageBuffer.done_writing_file(fname)

In [14]:
write_graph(15)

In [15]:
write_graph(10)

The write_graph function can be set as a callback to the time parameter, which will update the graph when the parameter changes, effectively injecting this computation into the C++ interaction.

In [16]:
time.register_callback(write_graph)

Time changed to 1630


In [17]:
time.clear_callbacks()

## Render disk buffers

CASM viewer exposes through the TINC server a json buffer that is used to update graphical markers that can show trajectories.

In [18]:
renderBuffer = tclient.get_disk_buffer('trajectory_buffer')

This disk buffer originates in a C++ TrajectoryRender. This disk buffer uses json to describe the positions and colors of vector markers. It is one of several rendering objects that are exposed to python through disk buffers.

In [19]:
pos = [[ [0.0,10, 0]], [ [0,5, 10]],[ [10,30, 0]], [ [-30, -10, 20]], [ [10, -10, 40]]]

In [20]:
def write_positions(pos):
    import json
    fname = renderBuffer.get_filename_for_writing()

    with open(fname, 'w') as f:
        json.dump(pos, f)
        renderBuffer.done_writing_file(fname)

In [21]:
write_positions(pos)

Rendering objects in TINC provide disk buffers as well as configuration options:

In [ ]:
tclient.get_parameter("width", "trajectory").value = 3.0

# Exploring datasets

This dataset is the result of multiple parameter sweeps, and TINC can help explore it easily by automating filesystem lookups. This can help you look up specific files that could be scattered across the filesystem. This is done through the ParameterSpace class that groups parameters as dimensions to the space.

In [ ]:
[ps.id for ps in tclient.parameter_spaces]

In [ ]:
ps=tclient.get_parameter_space("casmParameters")
ps.get_root_path()

In [ ]:
ps.get_current_realtive_path()

In [ ]:
ps.get_root_path() + ps.get_current_realtive_path()

In [ ]:
[(p.id, p.value) for p in ps.get_parameters()]

Changing the values of the 'dir' parameter in C++ will be sent here and will change the path for the parameter space

In [ ]:
[(p.id, p.value) for p in ps.get_parameters()]

In [ ]:
ps.get_root_path() + ps.get_current_realtive_path()

This path contains simulation results for specific parameter combinations, so you can explore the dataset across the multiple directories easily by using the sliders in C++ or programatically here in python:

In [ ]:
dir_param = ps.get_parameter('dir')
dir_param.value

In [ ]:
dir_param.value = 30
ps.get_root_path() + ps.get_current_realtive_path()

# Dynamic GUIs

Because you can create paramters in python as well as C++ you can dynamically create GUIs for the C++ application that trigger behavior defined in python

In [ ]:
from parameter import *

In [ ]:
eci1_param=tclient.create_parameter(Parameter,\
                                    "tet_oct_eci","casm",-0.375,0.375,\
                                    [-0.375,-0.125,0.125,0.375],-0.375)

In [ ]:
eci2_param=tclient.create_parameter(Parameter,\
                                    "oct_tet_NN","casm",2.0,6.0,[2.0,6.0],6.0)

In [ ]:
eci3_param=tclient.create_parameter(Parameter,\
                                    "oct_oct_NN","casm",0.0,1.0,\
                                    [0.0,1.0],0.0)

In [ ]:
eci4_param=tclient.create_parameter(Parameter,\
                                    "tet_tet_NN","casm",0.0,0.5,[0.0,0.5])

We can use these sliders to determine which dataset to load, by mapping their values to a path, and then setting a string parameter that is set up in C++ to load a dataset:

In [ ]:
root_dir = "C:/Users/Andres/source/repos/vdv_data/visualization/"
def create_dir_string_from_eci_param(eci_value):
    datasetname=root_dir + "AMX2_spinel_diffusion_0.0_0.0"+\
    "_"+str(eci1_param.value)+\
    "_"+str(eci2_param.value)+\
    "_"+str(eci3_param.value)+\
    "_"+str(eci4_param.value)+\
    "/kinetic_mc"
    tclient.get_parameter("dataset").value=datasetname
    print("Requested load: " + datasetname)
    return

By registering this callback to the parameters we just created, a change in any of them will trigger loading of a dataset.

In [ ]:
eci1_param.register_callback(create_dir_string_from_eci_param)

eci2_param.register_callback(create_dir_string_from_eci_param)

eci3_param.register_callback(create_dir_string_from_eci_param)

eci4_param.register_callback(create_dir_string_from_eci_param)

In [ ]:
eci1_param.value=0.125
tclient.wait_for_server_available()

In [ ]:
eci3_param.value=0.0
tclient.wait_for_server_available()

## Putting it all together

We can use all these features at once. We can create a gui button that triggers a python callback that will run a function that can use cache to update an image buffer. The effect for the user is that the graph will change when the button is pressed, but cache will be used if the function has already been called for that combination of parameters.

In [ ]:
button=tclient.create_parameter(Trigger,"trigger","casm")

In [ ]:
def cb(value):
    print("Hello!")

In [ ]:
button.register_callback(cb)